In [1]:
from src.baseline0 import build_baseline0_from_pickle

engine = build_baseline0_from_pickle("/Users/denis/Documents/COSC4397nlp/FinalProject/opinion_search/data/reviews_segment.pkl")
print(f"Index size: {len(engine.inverted_index)} unique terms")

# Test opinion query
results = engine.search("audio quality:poor", top_k=5)
for r in results:
    print(f"Score: {r.score} | {r.review.title}")

ModuleNotFoundError: No module named 'src'

In [ ]:
import sys
import os


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


from src.baseline0 import build_baseline0_from_pickle

In [2]:
# Test
engine = build_baseline0_from_pickle("../data/reviews_segment.pkl")
print(f"✓ Loaded {len(engine.reviews)} reviews")
print(f"✓ Index size: {len(engine.inverted_index)} unique terms")

# Quick test
results = engine.search("audio quality:poor", top_k=3)
print(f"\n✓ Found {len(results)} results for 'audio quality:poor'")
for i, r in enumerate(results, 1):
    print(f"  [{i}] Score={r.score} | {r.review.title[:60]}...")

✓ Loaded 210761 reviews
✓ Index size: 193770 unique terms

✓ Found 3 results for 'audio quality:poor'
  [1] Score=3 | 'MiBook ...MiBooHoo'...
  [2] Score=3 | 'Love It Already'...
  [3] Score=3 | 'Puke'...


In [ ]:
#required queries
test_queries = [
    "audio quality:poor",
    "wifi signal:strong", 
    "mouse button:click problem",
    "gps map:useful",
    "image quality:sharp"
]

print("="*60)
print("BASELINE0 - TEST QUERIES")
print("="*60)

for query in test_queries:
    results = engine.search(query, top_k=10)
    print(f"\nQuery: '{query}'")
    print(f"Retrieved: {len(results)} documents")
    
    if results:
        print("Top 3 results:")
        for i, r in enumerate(results[:3], 1):
            title = r.review.title[:50] if r.review.title else "<no title>"
            print(f"  [{i}] Score={r.score} | Rating={r.review.rating} | {title}")
    else:
        print("  No results found")

BASELINE0 - TEST QUERIES

Query: 'audio quality:poor'
Retrieved: 10 documents
Top 3 results:
  [1] Score=3 | Rating=2 | 'MiBook ...MiBooHoo'
  [2] Score=3 | Rating=5 | 'Love It Already'
  [3] Score=3 | Rating=2 | 'Puke'

Query: 'wifi signal:strong'
Retrieved: 10 documents
Top 3 results:
  [1] Score=3 | Rating=4 | 'Clear and strong signal'
  [2] Score=3 | Rating=5 | 'A great product!'
  [3] Score=3 | Rating=1 | 'ASUS ALREADY HAD IMPROVED SHELL DESIGN BEFORE REL

Query: 'mouse button:click problem'
Retrieved: 10 documents
Top 3 results:
  [1] Score=4 | Rating=2 | 'Adobe Acrobat Pro'
  [2] Score=4 | Rating=5 | 'comfortable and effective wireless mouse'
  [3] Score=4 | Rating=4 | 'Liked it so much I bought it three times... but n

Query: 'gps map:useful'
Retrieved: 10 documents
Top 3 results:
  [1] Score=3 | Rating=5 | 'The Best Book of Flight Simulator'
  [2] Score=3 | Rating=4 | 'Really useful improvement'
  [3] Score=3 | Rating=2 | 'User Interface Needs Serious Improvement'

Query: 'ima

In [4]:
from dataclasses import dataclass
from typing import List

@dataclass
class QueryEvaluation:
    query: str
    num_retrieved: int
    num_relevant: int
    precision: float
    
def evaluate_results_interactive(engine, query: str, top_k: int = 10):
    """Interactive evaluation helper - you manually mark relevant/irrelevant"""
    results = engine.search(query, top_k=top_k)
    
    print(f"\n{'='*70}")
    print(f"Evaluating Query: '{query}'")
    print(f"{'='*70}")
    
    if not results:
        print("No results found!")
        return QueryEvaluation(query, 0, 0, 0.0)
    
    relevant_count = 0
    
    for i, r in enumerate(results, 1):
        print(f"\n--- Result {i}/{len(results)} ---")
        print(f"Title: {r.review.title}")
        print(f"Rating: {r.review.rating} stars")
        print(f"Score: {r.score}")
        print(f"Text preview: {r.review.text[:300]}...")
        
        # Manual judgment
        while True:
            judgment = input("\nIs this RELEVANT? (y/n/q to quit): ").strip().lower()
            if judgment in ['y', 'n', 'q']:
                break
            print("Please enter 'y', 'n', or 'q'")
        
        if judgment == 'q':
            print("Evaluation stopped early.")
            break
        
        if judgment == 'y':
            relevant_count += 1
            print("✓ Marked as RELEVANT")
        else:
            print("✗ Marked as NOT RELEVANT")
    
    precision = relevant_count / len(results) if results else 0.0
    
    print(f"\n{'='*70}")
    print(f"Results for '{query}':")
    print(f"  Retrieved: {len(results)}")
    print(f"  Relevant: {relevant_count}")
    print(f"  Precision: {precision:.2%}")
    print(f"{'='*70}")
    
    return QueryEvaluation(query, len(results), relevant_count, precision)

In [5]:
result = evaluate_results_interactive(engine, "audio quality:poor", top_k=10)


Evaluating Query: 'audio quality:poor'

--- Result 1/10 ---
Title: 'MiBook ...MiBooHoo'
Rating: 2 stars
Score: 3
Text preview: 'Well, the concept is great. The visual and audio are very  poor quality. I guess we are spoiled with better quality in our gadgets. This reminds me of  a new ...first time....30 years ago gadget. Im so sad this gadget has a fuzzy visual and soft audio ( turn off your TV so you can hear it). Maybe m...
✓ Marked as RELEVANT

--- Result 2/10 ---
Title: 'Love It Already'
Rating: 5 stars
Score: 3
Text preview: 'I traded in my trusty Handspring Visor Pro of 4 years for my new T3.  I wasn\'t interested in the color screen, I was used to weeks of battery life, and it\'s a premium (priced) product.  However, a friend wanted it badly and was looking to me to be his technical support -- then I found out I could...
✗ Marked as NOT RELEVANT

--- Result 3/10 ---
Title: 'Puke'
Rating: 2 stars
Score: 3
Text preview: 'If you were a fan of Anthrax between 1985 and 1991 and sto

In [6]:
remaining_queries = [
    "wifi signal:strong",
    "mouse button:click problem", 
    "gps map:useful",
    "image quality:sharp"
]

baseline_results = []

# Add the audio quality result from above
baseline_results.append(QueryEvaluation("audio quality:poor", 10, 5, 0.50))

# Evaluate the rest
for query in remaining_queries:
    result = evaluate_results_interactive(engine, query, top_k=10)
    baseline_results.append(result)

# Print summary at the end
print("\n" + "="*70)
print("BASELINE0 - FINAL RESULTS SUMMARY")
print("="*70)
for result in baseline_results:
    print(f"{result.query:30} | Ret: {result.num_retrieved:2} | Rel: {result.num_relevant:2} | Prec: {result.precision:.1%}")


Evaluating Query: 'wifi signal:strong'

--- Result 1/10 ---
Title: 'Clear and strong signal'
Rating: 4 stars
Score: 3
Text preview: 'Just bought this DECT 6.0 system as my 4th generation of home cordless phones, after 900MHz, 2.4 GHz and 5.8 GHz systems of various manufacture. Signal is clear and strong through apartment walls and I like that there\'s guaranteed no interference with WiFi of any generation. Display is clear. Sett...
✓ Marked as RELEVANT

--- Result 2/10 ---
Title: 'A great product!'
Rating: 5 stars
Score: 3
Text preview: 'I\'ve been afraid to buy a router for the longest time. Before this I had the Linksys WRT54GL v1.1 with DD-WRT. Once Wireless N became more common, I decided I wanted to upgrade, but each router I looked at by Linksys looked cheap. Particularly the newer ones, without status LED\'s on the front. I ...
✗ Marked as NOT RELEVANT

--- Result 3/10 ---
Title: 'ASUS ALREADY HAD IMPROVED SHELL DESIGN BEFORE RELEASING PRIME'
Rating: 1 stars
Score: 3
Text previ